# Emulating live data

For more information, please visit this blog post on GeoNet -> 

In [4]:
#Please run each code cell in order! (CTRL + ENTER)

from arcgis.gis import GIS
from arcgis import features
from getpass import getpass #to accept passwords in an interactive fashion
from arcgis.features import FeatureLayer
import csv
import time
import getpass

In [5]:
#1. Insert your empty editable point feature layer with a "name" field. 
lyr_url = 'https://services.arcgis.com/Qo2anKIAMzIEkIJB/arcgis/rest/services/emulating_live_data/FeatureServer/0'
# You can use this as a test 

#2. Define the location of your .csv
csvLocation = 'data/SeanSamRichard.csv'

#3. Define how many people/items are updating in the csv. 
# More information regarding formatting the .csv scan be found on the cell below.
itemsPerIteration = 3

In [ ]:
#4. If your feature service is private, run this cell for an ArcGIS Online login prompt.
username = input("Enter your ArcGIS username ")   
password = getpass.getpass("Enter password: ")

gis = GIS("https://www.arcgis.com", username, password)
# You can also update the url above to your own ArcGIS Enterprise.

# If your feature service is public, ignore & you don't need to run this.

In [ ]:
#5. No editing needed - Just run this code! (Press CTRL + ENTER)

try:
  lyr_url
except NameError:
    print("ERROR -> PLEASE RUN YOUR VARIABLE CODE BLOCK FIRST (CTRL + ENTER)")
    #Tests one of the variables above to see if they've been initiated. 

layer = FeatureLayer(lyr_url)

def addFeature(name, long, lat):  
    point = {"attributes": 
        {"name": name},
        "geometry":
        {"x": long, "y": lat}}
    add_result = layer.edit_features(adds = [point])
    # This creates a point or points, and then adds them to the feature layer provided.
    
with open(csvLocation, newline='') as csvfile:   
    reader = csv.DictReader(csvfile)
    data = list(reader)
    row_count = len(data)
    dataCountForEachPerson = ((row_count)/itemsPerIteration)
    # Reads the provided CSV. It looks at how many lines are in your CSV and divides them by 'itemsPerIteration'.
    
    def updateFeatureLayer():
        # This loops through the csv and updates the features every 6 seconds.
        for x in range(0, int(dataCountForEachPerson)):
            layer.delete_features(where="1=1")
            for y in range(0, itemsPerIteration):
                name = data[x*itemsPerIteration+y]['name']
                long = int(data[x*itemsPerIteration+y]['longitude'])
                lat = int(data[x*itemsPerIteration+y]['latitude'])
                addFeature(name, long, lat)
                print( "name:", name, "longitude:", long, "latitude:", lat)
                if x*itemsPerIteration+y >= (row_count-1):
                    time.sleep(6) 
                    print("restarting in 6 seconds...")
                    updateFeatureLayer()
            print("Wait 6 seconds...")
            time.sleep(6) 
        
updateFeatureLayer()